This notebook demonstrates some limitations with OpenMC w.r.t. handling of decay-only nuclides in SaltProc. In Serpent, decay-only nuclides can be inlcuded in a material definition, and will be appropriately handled when running a burnup simulation. In OpenMC, only nuclides with cross section data can be included in a material, otherwise the program will raise an error. Due to this, decay-only nuclides that _should_ be in a material will start at zero after each processing step in SaltProc when coupled with OpenMC. I believe this is one of the factors that is causing our strange results for certain nuclides. 

I will now demonstrate this by loading depletion results for the first two SaltProc timesteps, and comparing nuclides in the OpenMC results to nuclides in the Serpent results

In [3]:
import openmc
from openmc.deplete import Results
import serpentTools

In [4]:
omc0 = Results('openmc_results/saltproc_runtime/step_0_data/depletion_results.h5')
omc1 = Results('openmc_results/saltproc_runtime/step_1_data/depletion_results.h5')

spn0 = serpentTools.read('serpent_results/saltproc_runtime/step_0_data/runtime_input.serpent_dep.m')
spn1 = serpentTools.read('serpent_results/saltproc_runtime/step_1_data/runtime_input.serpent_dep.m')

Consider the nuclide $^{245}Cm$. Based on the depletion chain I created, there are 4 ways to generate $^{245}Cm$:

1. $^{244}Cm (n,\gamma)^{245}Cm$, $Q=5520260.0$
2. $^{246}Cm (n,2n)^{245}Cm$, $Q=-6457580.0$
3. $^{247}Cm (n,3n)^{245}Cm$, $Q=-11613400.0$
4. $^{245}Am \rightarrow(\beta^-) ^{245}Cm$

Let's look at the amounts of all of these in our results files.

In [6]:
def side_by_side(nuc, omc, spn):
    omc_nuc = omc.get_mass('1', nuc, mass_units='g/cm3')[1]
    spn_nuc = spn.materials['fuel'].getValues('days', 'mdens', names=[nuc])
    print(f'OpenMC: {omc_nuc}')
    print(f'Serpent: {spn_nuc}')
    

In [25]:
print(omc0.get_mass('1', 'Am245', mass_units='g/cm3')[1])

[0.0000000e+00 2.4418282e-24]


In [26]:
for nuc in ['Cm244', 'Cm246', 'Cm247', 'Am245', 'Cm245']:
    print(nuc)
    side_by_side(nuc, omc0, spn0)
    print('\n')

Cm244
OpenMC: [4.05275732e-19 1.21513950e-18]
Serpent: [[4.05276e-19 1.21514e-18]]


Cm246
OpenMC: [4.08604235e-19 7.59518400e-19]
Serpent: [[4.08604e-19 7.59502e-19]]


Cm247
OpenMC: [4.10269973e-19 4.09102944e-19]
Serpent: [[4.10270e-19 4.09103e-19]]


Am245
OpenMC: [0.0000000e+00 2.4418282e-24]
Serpent: [[0.0000e+00 2.4466e-24]]


Cm245
OpenMC: [4.06940822e-19 5.44339652e-19]
Serpent: [[4.06941e-19 5.42227e-19]]




Now lets look at these in the next timestep

In [27]:
for nuc in ['Cm244', 'Cm246', 'Cm247', 'Am245', 'Cm245']:
    print(nuc)
    side_by_side(nuc, omc1, spn1)
    print('\n')

Cm244
OpenMC: [1.2151405e-18 1.2223320e-18]
Serpent: [[1.21514e-18 1.22233e-18]]


Cm246
OpenMC: [7.59519024e-19 9.07029443e-19]
Serpent: [[7.59503e-19 9.07552e-19]]


Cm247
OpenMC: [4.09103281e-19 4.07972973e-19]
Serpent: [[4.09103e-19 4.08065e-19]]


Am245
OpenMC: [0.00000000e+00 1.73435392e-24]
Serpent: [[2.44660e-24 1.75639e-24]]


Cm245
OpenMC: [5.44340100e-19 6.25980207e-19]
Serpent: [[5.42228e-19 6.27361e-19]]




In [36]:
i = 10
omc_far = Results(f'openmc_results/saltproc_runtime/step_{i}_data/depletion_results.h5')
spn_far = serpentTools.read(f'serpent_results/saltproc_runtime/step_{i}_data/runtime_input.serpent_dep.m')

In [37]:
for nuc in ['Cm244', 'Cm246', 'Cm247', 'Am245', 'Cm245']:
    print(nuc)
    side_by_side(nuc, omc_far, spn_far)
    print('\n')

Cm244
OpenMC: [1.25560119e-18 1.25951453e-18]
Serpent: [[1.25519e-18 1.25903e-18]]


Cm246
OpenMC: [1.18730953e-18 1.20330263e-18]
Serpent: [[1.18003e-18 1.19186e-18]]


Cm247
OpenMC: [3.99122653e-19 3.98029772e-19]
Serpent: [[3.99814e-19 3.98749e-19]]


Am245
OpenMC: [0.00000000e+00 1.73621512e-24]
Serpent: [[1.75492e-24 1.75685e-24]]


Cm245
OpenMC: [1.20774646e-18 1.39999650e-18]
Serpent: [[7.35280e-19 7.30136e-19]]




There is another issue where the OpenMC side of SaltProc is not storing these nulcides at all!

In [47]:
import saltproc

In [57]:
omc_res = saltproc.Results('openmc_results/saltproc_runtime/saltproc_results.h5')
spn_res = saltproc.Results('serpent_results/saltproc_runtime/saltproc_results.h5')

In [60]:
omc_res.get_nuclide_mass('fuel', 'Am245')

KeyError: 'Am245'

In [61]:
spn_res.get_nuclide_mass('fuel', 'Am245')

array([0.00000000e+00, 1.19173886e-16, 1.19173886e-16, 8.55537569e-17,
       8.55537569e-17, 8.52926713e-17, 8.52926713e-17, 8.54129850e-17,
       8.54129850e-17, 8.53365103e-17, 8.53365103e-17, 8.55084566e-17,
       8.55084566e-17, 8.52517549e-17, 8.52517549e-17, 8.53394329e-17,
       8.53394329e-17, 8.54378271e-17, 8.54378271e-17, 8.54821532e-17,
       8.54821532e-17, 8.55761635e-17, 8.55761635e-17, 8.55703183e-17,
       8.55703183e-17, 8.55292263e-17, 8.55292263e-17, 8.55217443e-17,
       8.55217443e-17, 8.57349172e-17, 8.57349172e-17, 8.58058572e-17,
       8.58058572e-17, 8.58129874e-17, 8.58129874e-17, 8.57071119e-17,
       8.57071119e-17, 8.59275890e-17, 8.59275890e-17, 8.58280459e-17,
       8.58280459e-17, 8.58161795e-17, 8.58161795e-17, 8.60678288e-17,
       8.60678288e-17, 8.57929339e-17, 8.57929339e-17, 8.59510614e-17,
       8.59510614e-17, 8.61652032e-17, 8.61652032e-17, 8.61586942e-17,
       8.61586942e-17, 8.59982663e-17, 8.59982663e-17, 8.60935583e-17,
      

The issue seems to step from `OpenMCDepcode.read_depleted_materials()`. Specifically, we should try to process materials from the `Results` object as much as possible instead of filtering everything through an `openmc.Material` object first, as this will remove nuclides that would otherwise be present in the SaltProc results file.

We will also need to figure out a way to copy the depletion results file from the most recent step, and update the nuclide compositions within so we can use `CoupledOperator.prev_res` to maintain the decay-only nuclides

It is possible that there is an issue with out depletion chain, but I'd like to first fix the SaltProc code issues. If results continue to underperform, we may want to try removing the Be7 evaluation as they do [here](https://github.com/openmc-dev/data/blob/master/depletion/generate_endf71_chain.py)